- [Mathematics for Deep Learning](http://www.d2l.ai/chapter_appendix-mathematics-for-deep-learning/index.html)

- 3-d: RGB 图片（宽 * 高 * 通道）
- 4-d: 一个 RGB 图片批量（批量大小 * 宽 * 高 * 通道）
- 5-d: 一个视频批量（批量大小 * 时间 * 宽 * 高 * 通道）

- 创建数组需要：

  - 形状
  - 每个元素的数据类型
  - 每个元素的值

- 访问元素子区域: `[::3, ::2]`
  - 这是一个 python 的切片操作，表示在**第一个**维度上，以**步长**3 进行切片，即取索引为 0、3、6、9 等的列
  - 在**第二个**维度上，以**步长**为 2 进行切片，即取索引为 0、2、4、6 等的列


In [1]:
import torch

In [2]:
x = torch.arange(12)

In [3]:
x.shape     # 查看形状

torch.Size([12])

In [4]:
x.numel()   # 元素的总数

12

In [5]:
x = x.reshape(3, 4)     # 改变一个张量的形状而不改变元素数量和元素值

In [7]:
torch.zeros((2, 3, 4))

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [8]:
torch.ones((2, 3, 4))

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

In [9]:
torch.tensor([[2, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])

tensor([[2, 1, 4, 3],
        [1, 2, 3, 4],
        [4, 3, 2, 1]])

In [10]:
x = torch.tensor([1.0, 2, 4, 8])    # 创建为浮点的tensor
y = torch.tensor([2, 2, 2, 2])      # 创建为整型的tensor
x + y, x - y, x / y, x**y

(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]),
 tensor([ 1.,  4., 16., 64.]))

In [11]:
torch.exp(x)

tensor([2.7183e+00, 7.3891e+00, 5.4598e+01, 2.9810e+03])

In [14]:
X = torch.arange(12, dtype=torch.float32).reshape((3, 4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)      # 结果显示两个

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

In [15]:
X == Y

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])

In [17]:
X.sum()

tensor(66.)

- 即使形状不同，我们仍然可以通过调用广播机制（broadcasting mechanism）来执行按元素操作


In [18]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

In [19]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

In [20]:
X[-1], X[1:3]   # 最后一行，1-2行

(tensor([ 8.,  9., 10., 11.]),
 tensor([[ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]))

In [26]:
X[0:2, :] = 12      # 为多个元素赋值相同的值
X

tensor([[12., 12., 12., 12.],
        [12., 12., 12., 12.],
        [ 8.,  9., 10., 11.]])

- 运行一些操作可能会导致为新结果分配内存


In [27]:
before = id(Y)
Y = Y + X
id(Y) == before

False

- 执行原地操作


In [28]:
Z = torch.zeros_like(Y)
print('id(Z):', id(Z))
Z[:] = X + Y
print('id(Z):', id(Z))

id(Z): 140707950984784
id(Z): 140707950984784


- 如果在后续计算中没有重复使用`X`，我们也可以使用`X[:] = X + Y` 或 `X += Y` 来减少操作的内存开销


In [29]:
before = id(X)
X += Y
id(X) == before

True

- 转换为 NumPy 张量，**NumPy 是数据处理经常使用的一个库**


In [30]:
A = X.numpy()
B = torch.tensor(A)
type(A), type(B)

(numpy.ndarray, torch.Tensor)

- 将大小为 1 的张量转换为 Python 标量


In [31]:
a = torch.tensor([3.5])
a, a.item(), float(a), int(a)

(tensor([3.5000]), 3.5, 3.5, 3)

---

### 数据预处理

- 创建一个人工数据集，并存储在 csv（逗号分隔值）文件


In [2]:
import os   # 用于处理文件和目录

os.makedirs(os.path.join('data'), exist_ok=True)    # 如果该文件夹已存在，则不会抛出错误
data_file = os.path.join('data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')   # 列名
    f.write('NA,Pave,127500\n')     # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

---

### 知识补充

CSV（Comma-Separated Values）文件是一种常见的文本文件格式，用于存储表格数据。在 CSV 文件中，每一行表示一个数据记录，每个数据记录由多个字段组成，字段之间使用逗号（`,`）或其他指定的分隔符进行分隔。

CSV 文件的基本特点包括：

1. **纯文本格式：** CSV 文件是纯文本文件，可以使用任何文本编辑器打开和编辑。

2. **逗号分隔字段：** CSV 文件中的数据字段之间使用逗号进行分隔。有时，也可以使用其他字符（如分号、制表符等）作为字段分隔符，取决于文件的规范。

3. **一行表示一个数据记录：** 每一行代表一个数据记录，记录中的字段按照特定的顺序进行排列。

4. **无数据类型：** CSV 文件中的数据都是以文本形式存储，没有固定的数据类型。因此，在读取 CSV 文件时，需要根据需要进行数据类型的转换。

CSV 文件通常用于存储结构简单、数据规模较小的表格数据。它在数据导出、数据交换和数据存储方面非常常见。许多电子表格软件（如 Microsoft Excel、Google Sheets 等）都支持导入和导出 CSV 文件。此外，CSV 文件也常用于在编程和数据分析中读取和处理数据，因为它是一种简单、易于处理的数据格式。

例如，下面是一个简单的 CSV 文件示例：

```csv
Name,Age,City
Alice,25,New York
Bob,30,San Francisco
Charlie,22,Los Angeles
```

在这个示例中，每行表示一个人的信息，包括姓名、年龄和所在城市。字段之间使用逗号进行分隔，第一行是列名，随后的每一行是一个数据记录。


- 从创建的 csv 文件中加载原始数据集


In [4]:
# 如果没有安装pandas，只需取消对以下行的注释：
# !pip install pandas
import pandas as pd

data = pd.read_csv(data_file)
print(data), data   # 在jupyter中可以不使用print，那么就是以html的形式进行输出

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


(None,
    NumRooms Alley   Price
 0       NaN  Pave  127500
 1       2.0   NaN  106000
 2       4.0   NaN  178100
 3       NaN   NaN  140000)

为了处理缺失的数据，典型的方法包括**插值**和**删除**，这里，我们将考虑**插值**

In [5]:
inputs, outputs = data.iloc[:, 0:2],data.iloc[:, 2]     # iloc 即 index location
inputs = inputs.fillna(inputs.mean())   # fillna 即 fill missing values
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


- 对于`inputs`中的类别值或离散值，我们将`NaN`视为一个类别
- 这里的`dummy_na=True`表示对原始数据中存在的缺失值（`NaN`），也会为其生成一个对应的One-Hot Encoding 向量，这样可以在模型中将缺失值作为一个独立的类别来处理
- `NaN`是"Not a Number"的缩写

In [6]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


- 现在`inputs` 和 `outputs` 中的所有条目都是数值类型，它们可以转换为张量格式

In [9]:
import torch

X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

> 上面的X的数据类型是float64，通常来讲会使用float32来加快计算的速度

In [10]:
# b 是 a 的一个view
a = torch.arange(12)
b = a.reshape((3,4))
b[:] = 2
a

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])